# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('NLP_ass_train.tsv', sep='\t')
val = pd.read_csv('NLP_ass_valid.tsv', sep='\t')
test = pd.read_csv('NLP_ass_test.tsv', sep='\t')

train.columns = ['text', 'label']
val.columns = ['text', 'label']
test.columns = ['text', 'label']

In [3]:
train

,text,label
0,nawt yall niggers ignoring me,normal
1,<user> i am bit confused coz chinese ppl can n...,hatespeech
2,and this is why i end up with nigger trainee d...,hatespeech
3,nogs jews and dykes how enriching,offensive
4,guilty until proven innocent unless you are a ...,hatespeech
...,...,...
15377,this vile racist ghetto ass dumber than a reta...,offensive
15378,when i first got on here and said i hate trump...,offensive
15379,was macht der moslem wenn der zion gegen seine...,normal
15380,it is awful look at world demographics asians ...,hatespeech


In [4]:
test

,text,label
0,laura loomer raped me while screaming at me in...,hatespeech
1,<user> what did the old lady do was she a nazi...,normal
2,as much as i appreciate a format of open discu...,normal
3,sex be so good a bitch be slow stroking and cr...,offensive
4,<user> <user> <user> with a room full of white...,normal
...,...,...
1918,it always women trying this shit like dingbat ...,offensive
1919,because women would never lie about being sexu...,offensive
1920,how is all that awesome muslim diversity going...,offensive
1921,well my dear lgbtq brothers and sisters i do n...,hatespeech


In [5]:
!pip install transformers[torch]

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   --------------------- ------------------ 1.3/2.4 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ------ --------------------------------- 1.6/10.0 MB 7.6 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.0 MB 6.9 MB/s eta 0:00:02
   ---------------- ----------------------- 4.2/10.0 MB 6.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.0 MB 6.6 MB/s eta 0:00:01
   -------------------------- ------------- 6.6/10.0 MB 6.4 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/10.0 MB 6.0 MB/s eta 0:00:01
   --------------------------------- ------ 8.4/10.0 MB 5.7 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/10.0 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.0 MB 5.3 MB/s eta 0:00:01
   ------------

# Model Loading

In [6]:
model_size = 'base'

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
import torch

model = AutoModelForSeq2SeqLM.from_pretrained(f"google/flan-t5-{model_size}")
tokenizer = AutoTokenizer.from_pretrained(f"google/flan-t5-{model_size}")
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ishaa\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admi

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

# Dataset Preparation

In [8]:
from string import Template
from torch.utils.data import Dataset, DataLoader

label2id = {
    'normal': 0,
    'hatespeech': 1,
    'offensive': 2
}

id2label = {
    0: 'normal',
    1: 'hatespeech',
    2: 'offensive'
}

option2id = {
    'A': 0,
    'B': 1,
    'C': 2
}


class HateSpeechDataset(Dataset):
  def __init__(self, data, tokenizer, prompt_type, fewshot, preamble, ans_type, options, template, device):
    self.text = list(data['text'])
    self.labels = list(data['label'].apply(lambda x: label2id[x]))
    self.prompt_type = prompt_type
    self.fewshot = fewshot
    self.ans_type = ans_type
    self.preamble = preamble
    self.options = options
    self.template = template
    self.tokenizer = tokenizer
    self.device = device

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    text = self.text[idx]
    label = self.labels[idx]

    if self.prompt_type:
      input_text = self.template.substitute(
          preamble=self.preamble, fewshot=self.fewshot, prompt=text, options=self.options
      )

    else:
      input_text = self.template.substitute(
          preamble=self.preamble, prompt=text, options=self.options
      )

    # print(input_text)
    encoding = self.tokenizer(
            input_text,
            return_tensors='pt'
        ).to(self.device)

    return encoding, label

In [9]:
prompt_type = 'zero_shot'
preamble = 'Is the sentence normal, hatespeech or offensive? Classify the sentence into one of these types - normal, hatespeech, offensive.'
options = ''
ans_type = 'label'
template = '$preamble\nSentence: $prompt\n$options\nAnswer:'
test_dataset = HateSpeechDataset(test, tokenizer, prompt_type, None, preamble, ans_type, options, Template(template), device)

In [10]:
test_dataset[0]

({'input_ids': tensor([[   27,     7,     8,  7142,  1389,     6,  5591,     7,   855, 10217,
             42, 12130,    58,  4501,  4921,     8,  7142,   139,    80,    13,
            175,  1308,     3,    18,  1389,     6,  5591,     7,   855, 10217,
              6, 12130,     5,  4892,    17,  1433,    10,    50,  2414,     3,
          14351,    49,     3,    52,     9,  3138,   140,   298, 25471,    44,
            140,    16,   160, 27635,    53,     3,   157,  5208,  1612,    11,
            243,    62,   398,  1215,  6544,   342,     8,   281,    63,   603,
             50,  2414,     3, 14351,    49,     3, 14351,    49,  5339, 11801,
             10,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 1)

In [11]:
templates = {}

In [12]:
prompt_types = ['zero_shot', 'few_shot']

for prompt_type in prompt_types:
  templates[prompt_type] = {
      'small': [],
      'base' : []
  }

In [13]:
def post_process(prediction, ans_type):
  if ans_type == 'label':
    if 'normal' in prediction.lower() or 'acceptable' in prediction.lower():
      return label2id['normal']
    elif 'hate' in prediction.lower():
      return label2id['hatespeech']
    elif 'offensive' in prediction.lower():
      return label2id['offensive']

  elif ans_type == 'option':
    if 'A' == prediction:
      return option2id['A']
    elif 'B' == prediction:
      return option2id['B']
    elif 'C' == prediction:
      return option2id['C']

# Zero-shot Inference

In [14]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from tqdm import tqdm

def inference(data):
  true_labels, pred_labels = [], []
  for i in tqdm(range(len(data))):
    input, label = data[i]
    output = model.generate(**input)
    pred = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    # print(pred)
    pred_label = post_process(pred, data.ans_type)
    true_labels.append(label)
    pred_labels.append(pred_label)

  assert len(true_labels) == len(pred_labels)
  num_correct = sum(1 for a, b in zip(pred_labels, true_labels) if a == b)
  test_acc = num_correct / len(true_labels)
  macro_f1 = f1_score(true_labels, pred_labels, average='macro')

  print(classification_report(true_labels, pred_labels))
  print(confusion_matrix(true_labels, pred_labels))

  return test_acc, macro_f1


test_acc, macro_f1 = inference(test_dataset)
print(f"\nTest acc : {test_acc:.4f}")
print(f"Test macro-F1 : {macro_f1:.4f}")

  0%|          | 0/1923 [00:00<?, ?it/s]c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1923/1923 [10:51<00:00,  2.95it/s]

              precision    recall  f1-score   support

           0       0.58      0.60      0.59       781
           1       0.45      0.78      0.57       594
           2       0.46      0.06      0.11       548

    accuracy                           0.50      1923
   macro avg       0.50      0.48      0.42      1923
weighted avg       0.51      0.50      0.45      1923

[[469 279  33]
 [121 466   7]
 [217 297  34]]

Test acc : 0.5039
Test macro-F1 : 0.4232


In [15]:
prompt_type = 'zero_shot'

In [16]:
templates[prompt_type][model_size].append(
    {
        'preamble': preamble,
        'options': options,
        'template': template,
        'ans_type': ans_type,
        'acc': test_acc,
        'macro_f1': macro_f1
    }
)

In [17]:
from pprint import pprint
pprint(templates)

{'few_shot': {'base': [], 'small': []},
 'zero_shot': {'base': [{'acc': 0.5039001560062403,
                         'ans_type': 'label',
                         'macro_f1': 0.42322900372911776,
                         'options': '',
                         'preamble': 'Is the sentence normal, hatespeech or '
                                     'offensive? Classify the sentence into '
                                     'one of these types - normal, hatespeech, '
                                     'offensive.',
                         'template': '$preamble\n'
                                     'Sentence: $prompt\n'
                                     '$options\n'
                                     'Answer:'}],
               'small': []}}


In [ ]:
import json
with open('templates.json', 'r') as f:
  json.dump(templates, f)

# Few-shot Inference

In [18]:
fewshot_prompts = [
    'Sentence: boatloads of refugee women children went missing in the mediterranean no explanations and how europe got flooded with nothing but middle east men at least those immigrants got to live others are not so lucky\nAnswer:normal',
    'Sentence: what your advice to a straight white conservative christian male with strong opinions trying to get a job\nAnswer:normal'
    'Sentence: either way kikes and sand niggers will most likely die this is very good\nAnswer:hatespeech',
    'Sentence: k nigger keep looking your daddy aint here try the white woman house down the corner\nAnswer:hatespeech',
    'Sentence: senile retarded democrat sounds about right\nAnswer:offensive',
    'Sentence: train guy as in the driver this is what happens when your base is mentally retarded and has mental issues\nAnswer:offensive',
]

fewshot = '\n\n'.join(fewshot_prompts)

In [19]:
fewshot

'Sentence: boatloads of refugee women children went missing in the mediterranean no explanations and how europe got flooded with nothing but middle east men at least those immigrants got to live others are not so lucky\nAnswer:normal\n\nSentence: what your advice to a straight white conservative christian male with strong opinions trying to get a job\nAnswer:normalSentence: either way kikes and sand niggers will most likely die this is very good\nAnswer:hatespeech\n\nSentence: k nigger keep looking your daddy aint here try the white woman house down the corner\nAnswer:hatespeech\n\nSentence: senile retarded democrat sounds about right\nAnswer:offensive\n\nSentence: train guy as in the driver this is what happens when your base is mentally retarded and has mental issues\nAnswer:offensive'

In [20]:
prompt_type = 'few_shot'
preamble = 'Is the sentence normal, hatespeech or offensive? Classify the sentence into one of these types - normal, hatespeech, offensive.'
options = ''
ans_type = 'label'
template = '$preamble\n$fewshot\n\nSentence: $prompt$options\nAnswer:'
test_dataset = HateSpeechDataset(test, tokenizer, prompt_type, fewshot, preamble, ans_type, options, Template(template), device)

In [21]:
test_dataset[0]

({'input_ids': tensor([[   27,     7,     8,  7142,  1389,     6,  5591,     7,   855, 10217,
             42, 12130,    58,  4501,  4921,     8,  7142,   139,    80,    13,
            175,  1308,     3,    18,  1389,     6,  5591,     7,   855, 10217,
              6, 12130,     5,  4892,    17,  1433,    10,  3432,  7134,     7,
             13, 14330,    15,   887,   502,   877,  3586,    16,     8,     3,
           5700, 12829,    29,    15,   152,   150,  7295,     7,    11,   149,
              3, 28188,   530,     3, 25895,    28,  1327,    68,  2214,  5727,
           1076,    44,   709,   273, 16096,   530,    12,   619,   717,    33,
             59,    78,  5722, 11801,    10, 12110,  4892,    17,  1433,    10,
            125,    39,  1867,    12,     3,     9,  2541,   872, 11252,     3,
          15294,    23,   152,  5069,    28,  1101,  8479,  1119,    12,   129,
              3,     9,   613, 11801,    10, 12110,   134,   295,  1433,    10,
            893,   194,   

In [22]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from tqdm import tqdm

def inference(data):
  true_labels, pred_labels = [], []
  for i in tqdm(range(len(data))):
    input, label = data[i]
    output = model.generate(**input)
    pred = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    # print(pred)
    pred_label = post_process(pred, data.ans_type)
    true_labels.append(label)
    pred_labels.append(pred_label)

  assert len(true_labels) == len(pred_labels)
  num_correct = sum(1 for a, b in zip(pred_labels, true_labels) if a == b)
  test_acc = num_correct / len(true_labels)
  macro_f1 = f1_score(true_labels, pred_labels, average='macro')

  print(classification_report(true_labels, pred_labels))
  print(confusion_matrix(true_labels, pred_labels))

  return test_acc, macro_f1


test_acc, macro_f1 = inference(test_dataset)
print(f"\nTest acc : {test_acc:.4f}")
print(f"Test macro-F1 : {macro_f1:.4f}")

  0%|          | 0/1923 [00:00<?, ?it/s]c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1923/1923 [13:05<00:00,  2.45it/s]

              precision    recall  f1-score   support

           0       0.60      0.58      0.59       781
           1       0.46      0.79      0.58       594
           2       0.42      0.12      0.19       548

    accuracy                           0.51      1923
   macro avg       0.50      0.50      0.45      1923
weighted avg       0.51      0.51      0.47      1923

[[452 259  70]
 [105 468  21]
 [201 280  67]]

Test acc : 0.5133
Test macro-F1 : 0.4539


In [ ]:
templates[prompt_type][model_size].append(
    {
        'preamble': preamble,
        'fewshot': fewshot,
        'options': options,
        'template': template,
        'ans_type': ans_type,
        'acc': test_acc,
        'macro_f1': macro_f1
    }
)

In [ ]:
from pprint import pprint
pprint(templates)

{'few_shot': {'base': [{'acc': 0.608,
                        'ans_type': 'label',
                        'fewshot': 'Sentence: tariq nasheed beating the fuck '
                                   'out of that drum beloved lmao somebody put '
                                   'a biscuit in that niggas mouth already\n'
                                   'Answer:normal\n'
                                   '\n'
                                   'Sentence: <number> <number> <percent> of '
                                   'brits think multiculturalism is a failure '
                                   'i guess the other <percent> are happy with '
                                   'their muzzie master there no great in '
                                   'britain anymore soon big ben will be '
                                   'converted into a minaret\n'
                                   'Answer:hatespeech\n'
                                   '\n'
                                  